## Overlap between green and red and the holotomographic channel

In [11]:
# Step 1: Import all required libraries
import pandas as pd
import numpy as np
from tifffile import imread, imsave
import cv2
from tkinter import filedialog as fd
import os
from datetime import datetime

# CORRECTED microscope calibration (microns per pixel)
um_per_pixel = 0.0876  # 87.6 nm per pixel

print("All libraries imported successfully!")
print(f"Using calibration: {um_per_pixel} μm/pixel ({um_per_pixel*1000} nm/pixel)")

All libraries imported successfully!
Using calibration: 0.0876 μm/pixel (87.6 nm/pixel)


In [12]:
# Step 2: File selection
print("Select the RED channel CSV file:")
red_csv = fd.askopenfilename(title="Select RED channel CSV")
print(f"Selected: {red_csv}")

print("Select the GREEN channel CSV file:")
green_csv = fd.askopenfilename(title="Select GREEN channel CSV") 
print(f"Selected: {green_csv}")

print("Select the holotomographic TIFF file:")
holo_tif = fd.askopenfilename(title="Select holotomographic TIFF")
print(f"Selected: {holo_tif}")

print("File selection complete!")

Select the RED channel CSV file:
Selected: Z:/Bisal_Halder_turbo/V601-Images-Armin/red_channel/condensates_AIO-img0_deconved-RNA.csv
Select the GREEN channel CSV file:
Selected: Z:/Bisal_Halder_turbo/V601-Images-Armin/green_channel/condensates_AIO-img0_deconved-gal9.csv
Select the holotomographic TIFF file:
Selected: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography/MAX_00_dec.tif
File selection complete!


In [14]:
# Step 3: Load and inspect the data
print("Loading data...")

# Load CSV files
df_red = pd.read_csv(red_csv)
df_green = pd.read_csv(green_csv)
print(f"Red channel CSV shape: {df_red.shape}")
print(f"Green channel CSV shape: {df_green.shape}")

# Load holotomographic image
holo_img = imread(holo_tif)
print(f"Holotomographic image shape: {holo_img.shape}")
print(f"Holotomographic image dtype: {holo_img.dtype}")
print(f"Holotomographic image intensity range: [{holo_img.min()}, {holo_img.max()}]")

# Display first few rows of data
print("\nRed channel data preview:")
display(df_red.head())
print("\nGreen channel data preview:")
display(df_green.head())

Loading data...
Red channel CSV shape: (333, 12)
Green channel CSV shape: (168, 12)
Holotomographic image shape: (1024, 1024)
Holotomographic image dtype: uint16
Holotomographic image intensity range: [0, 9244]

Red channel data preview:


,condensateID,contour_coord,center_x_pxl,center_y_pxl,area_um2,R_nm,mean_intensity,max_intensity,max_location,aspect_ratio,contour_solidity,contour_extent
0,1,"[[1016, 1012], [1015, 1013], [1014, 1013], [10...",1009.329268,1017.634146,1.683747,732.088325,849.965753,1230.0,"(1007, 1019)",1.416667,0.884892,0.602941
1,2,"[[272, 1009], [271, 1010], [270, 1010], [269, ...",271.696347,1014.358447,0.999297,563.991236,654.977273,902.0,"(272, 1015)",1.000000,0.948052,0.603306
2,3,"[[976, 1004], [975, 1005], [975, 1006], [974, ...",976.625850,1006.986395,0.335381,326.733728,319.617647,339.0,"(978, 1009)",0.857143,0.980000,0.583333
3,4,"[[880, 983], [879, 984], [878, 984], [877, 984...",878.184343,989.227273,1.806948,758.399243,509.085526,656.0,"(876, 991)",1.230769,0.967033,0.634615
4,5,"[[843, 969], [842, 970], [841, 970], [840, 971...",843.707763,973.577626,0.999297,563.991236,429.693182,627.0,"(844, 974)",1.300000,0.954248,0.561538



Green channel data preview:


,condensateID,contour_coord,center_x_pxl,center_y_pxl,area_um2,R_nm,mean_intensity,max_intensity,max_location,aspect_ratio,contour_solidity,contour_extent
0,1,"[[75, 896], [74, 897], [74, 898], [75, 899], [...",75.577778,897.577778,0.102667,180.776327,176.384615,256.0,"(76, 898)",1.000000,1.000000,0.468750
1,2,"[[361, 893], [360, 894], [360, 895], [359, 896...",362.403226,896.032258,0.424359,367.529135,318.975610,912.0,"(362, 896)",1.142857,0.953846,0.553571
2,3,"[[354, 884], [353, 885], [353, 886], [352, 887...",355.264368,886.931034,0.396981,355.475705,348.707317,661.0,"(356, 886)",1.142857,0.852941,0.517857
3,4,"[[366, 879], [365, 880], [364, 881], [364, 882...",368.177264,884.148362,1.184099,613.930174,471.679612,3104.0,"(368, 884)",1.000000,0.982955,0.714876
4,5,"[[400, 855], [399, 856], [398, 856], [397, 857...",396.879937,862.347551,1.444190,678.011648,336.629921,1219.0,"(398, 860)",1.066667,0.857724,0.439583


In [23]:
# Step 4: Create and save thresholded holotomographic mask
print("Creating holotomographic mask...")

# Create binary mask for holotomographic signals (intensity > 800)
holo_mask = (holo_img > 800).astype(np.uint8) * 255

print(f"Holotomographic mask shape: {holo_mask.shape}")
print(f"Pixels above threshold: {np.sum(holo_mask > 0)}")
print(f"Percentage of image above threshold: {np.sum(holo_mask > 0) / holo_mask.size * 100:.2f}%")

# Show mask properties
print(f"Mask unique values: {np.unique(holo_mask)}")

# Save only the version with original image values above threshold in the holotomography folder
holo_thresholded_values = np.where(holo_img > 800, holo_img, 0)

# Save in the same folder as the input holotomographic image
holo_dir = os.path.dirname(holo_tif)
thresholded_values_filename = os.path.join(holo_dir, "holo_values_above_800.tif")
imsave(thresholded_values_filename, holo_thresholded_values)
print(f"✓ Holotomographic values above 800 saved to: {thresholded_values_filename}")

print("\nYou can now open this TIFF file in ImageJ to verify the thresholding results:")
print(f"1. {thresholded_values_filename} - Original intensity values (0 for values ≤800)")

Creating holotomographic mask...
Holotomographic mask shape: (1024, 1024)
Pixels above threshold: 37825
Percentage of image above threshold: 3.61%
Mask unique values: [  0 255]
✓ Holotomographic values above 800 saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\holo_values_above_800.tif

You can now open this TIFF file in ImageJ to verify the thresholding results:
1. Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\holo_values_above_800.tif - Original intensity values (0 for values ≤800)


C:\Users\bisal\AppData\Local\Temp\4\ipykernel_17756\1417528937.py:20: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(thresholded_values_filename, holo_thresholded_values)


In [24]:
# Step 5: Define helper functions to count overlapping condensates
def count_overlapping_condensates(df, holo_mask, channel_name):
    """Count how many condensates overlap with holotomographic signals"""
    total_condensates = len(df)
    overlapping_condensates = 0
    individual_results = []
    
    print(f"\nProcessing {channel_name} channel: {total_condensates} condensates")
    
    for idx, row in df.iterrows():
        try:
            # Recreate condensate mask from contour coordinates
            contour_coords = eval(row['contour_coord'])
            contour_array = np.array(contour_coords, dtype=np.int32).reshape(-1, 1, 2)
            
            # Create mask for this condensate
            condensate_mask = np.zeros(holo_mask.shape, dtype=np.uint8)
            cv2.fillPoly(condensate_mask, [contour_array], 255)
            
            # Calculate overlap with holo signals
            overlap_mask = cv2.bitwise_and(condensate_mask, holo_mask)
            overlap_pixels = np.sum(overlap_mask > 0)
            
            # Check if this condensate overlaps with holo signals (at least 1 pixel overlap)
            does_overlap = overlap_pixels > 0
            
            if does_overlap:
                overlapping_condensates += 1
            
            individual_results.append({
                'condensate_id': row['condensateID'],
                'channel': channel_name,
                'overlaps_with_holo': does_overlap,
                'overlap_pixels': overlap_pixels,
                'area_um2': row['area_um2'],
                'R_nm': row['R_nm']
            })
            
            if idx < 5:  # Show first 5 for debugging
                status = "OVERLAPS" if does_overlap else "no overlap"
                print(f"  Condensate {idx+1}: {status} ({overlap_pixels} overlap pixels)")
                
        except Exception as e:
            print(f"Error processing {channel_name} condensate {idx+1}: {e}")
            continue
    
    if total_condensates > 0:
        overlap_percentage = (overlapping_condensates / total_condensates) * 100
    else:
        overlap_percentage = 0
        
    print(f"  Total {channel_name} condensates: {total_condensates}")
    print(f"  Overlapping with holo signals: {overlapping_condensates}")
    print(f"  Overlap percentage: {overlap_percentage:.2f}%")
        
    return overlap_percentage, overlapping_condensates, total_condensates, individual_results

def count_combined_overlap(df_red, df_green, holo_mask):
    """Count condensates that are in BOTH red and green channels AND overlap with holo signals"""
    print("\nCalculating combined overlap...")
    
    combined_overlap_count = 0
    total_combined_condensates = 0
    
    # Check each red condensate to see if it overlaps with any green condensate AND holo signals
    for red_idx, red_row in df_red.iterrows():
        try:
            # Create red condensate mask
            red_contour_coords = eval(red_row['contour_coord'])
            red_contour_array = np.array(red_contour_coords, dtype=np.int32).reshape(-1, 1, 2)
            red_mask = np.zeros(holo_mask.shape, dtype=np.uint8)
            cv2.fillPoly(red_mask, [red_contour_array], 255)
            
            # Check if this red condensate overlaps with any green condensate
            overlaps_with_green = False
            for green_idx, green_row in df_green.iterrows():
                try:
                    # Create green condensate mask
                    green_contour_coords = eval(green_row['contour_coord'])
                    green_contour_array = np.array(green_contour_coords, dtype=np.int32).reshape(-1, 1, 2)
                    green_mask = np.zeros(holo_mask.shape, dtype=np.uint8)
                    cv2.fillPoly(green_mask, [green_contour_array], 255)
                    
                    # Check overlap between red and green
                    red_green_overlap = cv2.bitwise_and(red_mask, green_mask)
                    if np.sum(red_green_overlap > 0) > 0:
                        overlaps_with_green = True
                        break
                except Exception as e:
                    continue
            
            if overlaps_with_green:
                total_combined_condensates += 1
                # Check if this red+green overlapping condensate also overlaps with holo
                red_holo_overlap = cv2.bitwise_and(red_mask, holo_mask)
                if np.sum(red_holo_overlap > 0) > 0:
                    combined_overlap_count += 1
                    
        except Exception as e:
            print(f"Error processing red condensate {red_idx+1}: {e}")
            continue
    
    print(f"  Total red+green overlapping condensates: {total_combined_condensates}")
    print(f"  Overlapping with holo signals: {combined_overlap_count}")
    
    if total_combined_condensates > 0:
        combined_overlap_percentage = (combined_overlap_count / total_combined_condensates) * 100
    else:
        combined_overlap_percentage = 0
        
    return combined_overlap_percentage, combined_overlap_count, total_combined_condensates

print("Helper functions defined!")

Helper functions defined!


In [25]:
# Step 6: Calculate overlaps
print("Starting overlap calculations...")
print("="*50)

# Count overlapping condensates for each channel
red_overlap_pct, red_overlap_count, red_total_count, red_individual = count_overlapping_condensates(df_red, holo_mask, "RED")
green_overlap_pct, green_overlap_count, green_total_count, green_individual = count_overlapping_condensates(df_green, holo_mask, "GREEN")
both_overlap_pct, both_overlap_count, both_total_count = count_combined_overlap(df_red, df_green, holo_mask)

print("\n" + "="*50)
print("OVERLAP ANALYSIS RESULTS")
print("="*50)
print(f"RED channel: {red_overlap_pct:.2f}% ({red_overlap_count}/{red_total_count}) overlap with holo signals")
print(f"GREEN channel: {green_overlap_pct:.2f}% ({green_overlap_count}/{green_total_count}) overlap with holo signals")
print(f"BOTH channels: {both_overlap_pct:.2f}% ({both_overlap_count}/{both_total_count}) overlap with holo signals")

Starting overlap calculations...

Processing RED channel: 333 condensates
  Condensate 1: no overlap (0 overlap pixels)
  Condensate 2: no overlap (0 overlap pixels)
  Condensate 3: no overlap (0 overlap pixels)
  Condensate 4: OVERLAPS (5 overlap pixels)
  Condensate 5: no overlap (0 overlap pixels)
  Total RED condensates: 333
  Overlapping with holo signals: 141
  Overlap percentage: 42.34%

Processing GREEN channel: 168 condensates
  Condensate 1: OVERLAPS (6 overlap pixels)
  Condensate 2: no overlap (0 overlap pixels)
  Condensate 3: OVERLAPS (2 overlap pixels)
  Condensate 4: no overlap (0 overlap pixels)
  Condensate 5: OVERLAPS (15 overlap pixels)
  Total GREEN condensates: 168
  Overlapping with holo signals: 75
  Overlap percentage: 44.64%

Calculating combined overlap...
  Total red+green overlapping condensates: 61
  Overlapping with holo signals: 45

OVERLAP ANALYSIS RESULTS
RED channel: 42.34% (141/333) overlap with holo signals
GREEN channel: 44.64% (75/168) overlap wit

In [26]:
# Step 7: Create and save results + visualization
print("Creating results dataframes and visualization...")

# Create summary results
summary_data = {
    'analysis_type': ['Red Channel', 'Green Channel', 'Both Channels Overlap'],
    'overlap_percentage': [red_overlap_pct, green_overlap_pct, both_overlap_pct],
    'overlapping_condensates': [red_overlap_count, green_overlap_count, both_overlap_count],
    'total_condensates': [red_total_count, green_total_count, both_total_count],
    'holo_threshold': [800, 800, 800],
    'um_per_pixel': [um_per_pixel, um_per_pixel, um_per_pixel],
    'timestamp': [datetime.now().strftime("%Y-%m-%d %H:%M:%S")] * 3
}

summary_df = pd.DataFrame(summary_data)

# Create detailed individual results
individual_df = pd.DataFrame(red_individual + green_individual)

print("Summary dataframe:")
display(summary_df)
print(f"\nDetailed dataframe shape: {individual_df.shape}")
print("Detailed dataframe preview:")
display(individual_df.head())

# Create separate visualization TIFF files - Save in holotomography folder
print("\n" + "="*50)
print("CREATING SEPARATE VISUALIZATION TIFF FILES")
print("="*50)

def create_separate_visualizations(df_red, df_green, holo_img, holo_mask, output_dir):
    # Case 1: Red + Holo overlap
    red_holo_vis = np.zeros((holo_img.shape[0], holo_img.shape[1], 3), dtype=np.uint8)
    # Add holotomography in blue
    red_holo_vis[:, :, 2] = (holo_mask > 0).astype(np.uint8) * 255
    # Add red condensates
    for idx, row in df_red.iterrows():
        try:
            contour_coords = eval(row['contour_coord'])
            contour_array = np.array(contour_coords, dtype=np.int32).reshape(-1, 1, 2)
            cv2.fillPoly(red_holo_vis, [contour_array], (255, 0, 0))
        except Exception as e:
            continue
    red_holo_filename = os.path.join(output_dir, "red_holo_overlap.tif")
    imsave(red_holo_filename, red_holo_vis)
    print(f"✓ Red + Holo overlap saved to: {red_holo_filename}")
    
    # Case 2: Green + Holo overlap
    green_holo_vis = np.zeros((holo_img.shape[0], holo_img.shape[1], 3), dtype=np.uint8)
    # Add holotomography in blue
    green_holo_vis[:, :, 2] = (holo_mask > 0).astype(np.uint8) * 255
    # Add green condensates
    for idx, row in df_green.iterrows():
        try:
            contour_coords = eval(row['contour_coord'])
            contour_array = np.array(contour_coords, dtype=np.int32).reshape(-1, 1, 2)
            cv2.fillPoly(green_holo_vis, [contour_array], (0, 255, 0))
        except Exception as e:
            continue
    green_holo_filename = os.path.join(output_dir, "green_holo_overlap.tif")
    imsave(green_holo_filename, green_holo_vis)
    print(f"✓ Green + Holo overlap saved to: {green_holo_filename}")
    
    # Case 3: Red + Green + Holo overlap
    all_three_vis = np.zeros((holo_img.shape[0], holo_img.shape[1], 3), dtype=np.uint8)
    # Add holotomography in blue
    all_three_vis[:, :, 2] = (holo_mask > 0).astype(np.uint8) * 255
    # Add red condensates
    for idx, row in df_red.iterrows():
        try:
            contour_coords = eval(row['contour_coord'])
            contour_array = np.array(contour_coords, dtype=np.int32).reshape(-1, 1, 2)
            cv2.fillPoly(all_three_vis, [contour_array], (255, 0, 0))
        except Exception as e:
            continue
    # Add green condensates
    for idx, row in df_green.iterrows():
        try:
            contour_coords = eval(row['contour_coord'])
            contour_array = np.array(contour_coords, dtype=np.int32).reshape(-1, 1, 2)
            cv2.fillPoly(all_three_vis, [contour_array], (0, 255, 0))
        except Exception as e:
            continue
    all_three_filename = os.path.join(output_dir, "red_green_holo_overlap.tif")
    imsave(all_three_filename, all_three_vis)
    print(f"✓ Red + Green + Holo overlap saved to: {all_three_filename}")
    
    return red_holo_filename, green_holo_filename, all_three_filename

# Use holotomography folder for all outputs
holo_dir = os.path.dirname(holo_tif)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save CSV files in holotomography folder
summary_filename = os.path.join(holo_dir, f"condensate_overlap_summary_{timestamp}.csv")
summary_df.to_csv(summary_filename, index=False)
print(f"✓ Summary results saved to: {summary_filename}")

individual_filename = os.path.join(holo_dir, f"condensate_overlap_detailed_{timestamp}.csv")
individual_df.to_csv(individual_filename, index=False)
print(f"✓ Detailed results saved to: {individual_filename}")

# Create visualization files in holotomography folder
red_holo_file, green_holo_file, all_three_file = create_separate_visualizations(df_red, df_green, holo_img, holo_mask, holo_dir)

print("\n" + "="*50)
print("ALL FILES SAVED IN HOLOTOMOGRAPHY FOLDER")
print("="*50)
print(f"CSV Files:")
print(f"1. {os.path.basename(summary_filename)}")
print(f"2. {os.path.basename(individual_filename)}")
print(f"\nVisualization TIFF Files:")
print(f"1. red_holo_overlap.tif - Red fluorescence + Holotomography")
print(f"2. green_holo_overlap.tif - Green fluorescence + Holotomography") 
print(f"3. red_green_holo_overlap.tif - All three channels together")
print(f"4. holo_values_above_800.tif - Holotomography threshold verification")
print(f"\nAll files saved to: {holo_dir}")

Creating results dataframes and visualization...
Summary dataframe:


,analysis_type,overlap_percentage,overlapping_condensates,total_condensates,holo_threshold,um_per_pixel,timestamp
0,Red Channel,42.342342,141,333,800,0.0876,2025-10-13 17:30:58
1,Green Channel,44.642857,75,168,800,0.0876,2025-10-13 17:30:58
2,Both Channels Overlap,73.770492,45,61,800,0.0876,2025-10-13 17:30:58



Detailed dataframe shape: (501, 6)
Detailed dataframe preview:


,condensate_id,channel,overlaps_with_holo,overlap_pixels,area_um2,R_nm
0,1,RED,False,0,1.683747,732.088325
1,2,RED,False,0,0.999297,563.991236
2,3,RED,False,0,0.335381,326.733728
3,4,RED,True,5,1.806948,758.399243
4,5,RED,False,0,0.999297,563.991236



CREATING SEPARATE VISUALIZATION TIFF FILES
✓ Summary results saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\condensate_overlap_summary_20251013_173058.csv
✓ Detailed results saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\condensate_overlap_detailed_20251013_173058.csv


C:\Users\bisal\AppData\Local\Temp\4\ipykernel_17756\1729939107.py:45: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(red_holo_filename, red_holo_vis)
C:\Users\bisal\AppData\Local\Temp\4\ipykernel_17756\1729939107.py:61: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(green_holo_filename, green_holo_vis)


✓ Red + Holo overlap saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\red_holo_overlap.tif
✓ Green + Holo overlap saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\green_holo_overlap.tif
✓ Red + Green + Holo overlap saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography\red_green_holo_overlap.tif

ALL FILES SAVED IN HOLOTOMOGRAPHY FOLDER
CSV Files:
1. condensate_overlap_summary_20251013_173058.csv
2. condensate_overlap_detailed_20251013_173058.csv

Visualization TIFF Files:
1. red_holo_overlap.tif - Red fluorescence + Holotomography
2. green_holo_overlap.tif - Green fluorescence + Holotomography
3. red_green_holo_overlap.tif - All three channels together
4. holo_values_above_800.tif - Holotomography threshold verification

All files saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/holotomography


C:\Users\bisal\AppData\Local\Temp\4\ipykernel_17756\1729939107.py:85: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(all_three_filename, all_three_vis)


In [28]:
# Step 8: Save to CSV files
print("Saving results to CSV files...")

output_dir = os.path.dirname(red_csv)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Summary CSV
summary_filename = os.path.join(output_dir, f"condensate_overlap_summary_{timestamp}.csv")
summary_df.to_csv(summary_filename, index=False)
print(f"✓ Summary results saved to: {summary_filename}")

# Detailed individual results CSV
individual_filename = os.path.join(output_dir, f"condensate_overlap_detailed_{timestamp}.csv")
individual_df.to_csv(individual_filename, index=False)
print(f"✓ Detailed results saved to: {individual_filename}")

print("\n" + "="*50)
print("ANALYSIS COMPLETE!")
print("="*50)
print(f"Red channel: {red_overlap_pct:.2f}% ({red_overlap_count}/{red_total_count}) overlap with holo signals")
print(f"Green channel: {green_overlap_pct:.2f}% ({green_overlap_count}/{green_total_count}) overlap with holo signals")
print(f"Both channels: {both_overlap_pct:.2f}% ({both_overlap_count}/{both_total_count}) overlap with holo signals")
print(f"\nTotal red condensates analyzed: {red_total_count}")
print(f"Total green condensates analyzed: {green_total_count}")
print(f"Holo signal threshold: >800 intensity")
print(f"Pixel calibration: {um_per_pixel} μm/pixel ({um_per_pixel*1000} nm/pixel)")
print(f"\nAll files saved to: {output_dir}")

Saving results to CSV files...
✓ Summary results saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/red_channel\condensate_overlap_summary_20251013_175849.csv
✓ Detailed results saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/red_channel\condensate_overlap_detailed_20251013_175849.csv

ANALYSIS COMPLETE!
Red channel: 42.34% (141/333) overlap with holo signals
Green channel: 44.64% (75/168) overlap with holo signals
Both channels: 73.77% (45/61) overlap with holo signals

Total red condensates analyzed: 333
Total green condensates analyzed: 168
Holo signal threshold: >800 intensity
Pixel calibration: 0.0876 μm/pixel (87.6 nm/pixel)

All files saved to: Z:/Bisal_Halder_turbo/V601-Images-Armin/red_channel
